In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
page_num = "page/{num}/"

In [3]:
ai_url = "https://neurosciencenews.com/neuroscience-topics/artificial-intelligence-2/"
robotics_url = "https://neurosciencenews.com/neuroscience-topics/robotics-2/"
psychology_url = "https://neurosciencenews.com/neuroscience-topics/psychology/"
neurology_url = "https://neurosciencenews.com/neuroscience-topics/neurology/"
neuroscience_url = "https://neurosciencenews.com/neuroscience-topics/neuroscience/"

In [2]:
url = "https://neurosciencenews.com/moving-object-background-14424/"

In [4]:
ai_pages = 30
robotics_pages = 17
psychology_pages = 313
neurology_pages = 296
neuroscience_pages = 543

In [6]:
r = requests.get("https://neurosciencenews.com/neuroscience-topics/artificial-intelligence-2/")
raw_html = r.content
soup_html = BeautifulSoup(raw_html, "html.parser")

In [22]:
count = 1
for box in soup_html.findAll("div", class_="cb-meta clearfix"):
    print(box.time.text)
    print(count, box.a['href'])
    print('------')
    count += 1

July 3, 2019
1 https://neurosciencenews.com/ai-aging-longevity-14412/
------
June 29, 2019
2 https://neurosciencenews.com/prosthetic-rock-paper-scissors-14373/
------
June 22, 2019
3 https://neurosciencenews.com/deepfakes-facial-quirks-14298/
------
June 22, 2019
4 https://neurosciencenews.com/concept-comprehension-14296/
------
June 21, 2019
5 https://neurosciencenews.com/clutter-objects-14295/
------
June 20, 2019
6 https://neurosciencenews.com/mirobiome-fibromyalgia-14286/
------
June 20, 2019
7 https://neurosciencenews.com/bci-prosthetic-arm-14283/
------
June 19, 2019
8 https://neurosciencenews.com/ai-alexa-heart-monitor-14275/
------
June 19, 2019
9 https://neurosciencenews.com/medical-ai-14274/
------
June 18, 2019
10 https://neurosciencenews.com/face-identity-information-14255/
------
June 13, 2019
11 https://neurosciencenews.com/ai-psychosis-words-14236/
------
June 12, 2019
12 https://neurosciencenews.com/ai-multiple-sclerosis-treatment-14225/
------


In [10]:
for page in soup_html.findAll("h2", class_="cb-post-title"):
    print(page)
    print("-------------")

<h2 class="cb-post-title"><a href="https://neurosciencenews.com/brain-disease-neuroscience-collaboration-14427/">To cure brain diseases, neuroscientists must collaborate: That’s why I’m giving my data away</a></h2>
-------------
<h2 class="cb-post-title"><a href="https://neurosciencenews.com/roles-performance-14426/">Sister, neighbor, friend: Awareness of multiple roles boosts kids’ performance</a></h2>
-------------
<h2 class="cb-post-title"><a href="https://neurosciencenews.com/public-health-challenge-anti-vaxxers-14425/">Top global public health scientists launch new challenge to anti-vaxxers</a></h2>
-------------
<h2 class="cb-post-title"><a href="https://neurosciencenews.com/brain-disease-neuroscience-collaboration-14427/">To cure brain diseases, neuroscientists must collaborate: That’s why I’m giving my data away</a></h2>
-------------
<h2 class="cb-post-title"><a href="https://neurosciencenews.com/roles-performance-14426/">Sister, neighbor, friend: Awareness of multiple roles b